In [1]:
import os
import re
import time
import json
import base64
import serial
import hashlib

In [2]:
COM_PORT = "COM10"
BAUD_RATE = 115200

serial_port = serial.Serial()
serial_port.baudrate = BAUD_RATE
serial_port.port = COM_PORT
serial_port.timeout = 10
serial_port.open()

In [3]:
def reset_board(serial_port_obj: serial.Serial):
    if not serial_port_obj and not serial_port_obj.is_open:
        return
    """Reset the board by sending a soft reset command over serial. ONLY WORKS with MicroPython."""
    pythonInject = [
        "import machine",
        "machine.reset()",
    ]
    # interrupt the currently running code
    serial_port_obj.write(b"\x03")  # Ctrl+C
    # just consumed all the output until the prompt, try up to 10 seconds
    time_start = time.time()
    while time.time() - time_start < 10:
        line = serial_port_obj.readline().decode("utf-8").strip()
        if "MicroPython v" in line:
            break
    serial_port_obj.write(b"\x01")  # switch to raw REPL mode & inject code
    time.sleep(0.1)
    time_start = time.time()
    while time.time() - time_start < 10:
        line = serial_port_obj.readline().decode("utf-8").strip()
        if ">>>" in line:
            break
    for code in pythonInject:
        serial_port_obj.write(bytes(code + "\n", "utf-8"))
    serial_port_obj.write(b"\x04")  # exit raw REPL and run injected code
    serial_port_obj.close()
    time.sleep(1)


reset_board(serial_port_obj=serial_port)

In [4]:
serial_port.open()

In [5]:
# request available space on the disk if requested by sending a json with request_disc_available_space as true
def request_disc_available_space(serial_port) -> tuple[int, int]:
    header = {
        "request_disc_available_space": True,
    }
    message = (json.dumps(header) + "\n").encode()
    serial_port.write(message)
    while True:
        try:
            response = serial_port.readline().decode().strip()
            print("Device -> PC:", response)
            if "Info: Available space" in response:
                # return format "Device -> PC: Info: Available space on the disk: 1228800 bytes, total space: 1441792 bytes."
                match = re.search(r"(\d+) bytes, total space: (\d+) bytes", response)
                if match:
                    available_space = int(match.group(1))
                    total_space = int(match.group(2))
                    return available_space, total_space
        except Exception as e:
            print(f"Error: {e}")
            break
    return 0, 0


available_space, total_space = request_disc_available_space(serial_port)

Device -> PC: Info: Available space on the disk: 294912 bytes, total space: 1441792 bytes.


In [6]:
# request all files stat on the disk if requested by sending a json with request_dir_list as true
def request_dir_list(serial_port) -> dict:
    header = {
        "request_dir_list": True,
    }
    message = (json.dumps(header) + "\n").encode()
    serial_port.write(message)
    while True:
        try:
            response = serial_port.readline().decode().strip()
            # print("Device -> PC:", response)
            if "Info: Directory list" in response:
                # return format "Device -> PC: Info: Directory list: "start of a directory"
                match = re.search(r"Info: Directory list: (.*)$", response)
                if match:
                    output = json.loads(match.group(1))
                    return output
        except Exception as e:
            print(f"Error: {e}")
            break
    return {}


file_stats = request_dir_list(serial_port)
# the last 4th field in the list is the file size
file_stats_size_only = {file: stat[-4] for file, stat in file_stats.items()}
file_stats_size_only

{'autosampler_config.json': 311,
 'pumps_config.json': 815,
 'bootloader_util.py': 1267,
 'pump_control_pico.py': 19838,
 'main.py': 111,
 'bootloader.py': 7464,
 'pwm_dma_fade_onetime.py': 5955,
 'autosampler_status.txt': 11,
 'test.pdf': 1052542,
 'bootloader_config.json': 62,
 'frequency_counter_reciprocal_with_interrupt.py': 9351,
 'autosampler_control_pico.py': 14068}

In [7]:
# request avilable memory on the device if requested by sending a json with request_memory as true
def request_memory(serial_port) -> tuple[int, int]:
    header = {
        "request_memory": True,
    }
    message = (json.dumps(header) + "\n").encode()
    serial_port.write(message)
    while True:
        try:
            response = serial_port.readline().decode().strip()
            print("Device -> PC:", response)
            if "Info: free Memory" in response:
                # return format "Device -> PC: Info: free Memory 218864 bytes, total Memory 233024 bytes."
                match = re.search(r"(\d+) bytes, total Memory (\d+) bytes", response)
                if match:
                    free_memory = int(match.group(1))
                    total_memory = int(match.group(2))
                    return free_memory, total_memory
        except Exception as e:
            print(f"Error: {e}")
            break
    return 0, 0


free_memory, total_memory = request_memory(serial_port)

Device -> PC: Info: free Memory 218480 bytes, total Memory 233024 bytes.


In [8]:
def remove_file(serial_port, file_path) -> bool:
    header = {
        "remove_file_request": True,
        "filename": file_path,
    }
    message = (json.dumps(header) + "\n").encode()
    serial_port.write(message)
    while True:
        try:
            response = serial_port.readline().decode().strip()
            print("Device -> PC:", response)
            if "Success: Removed file" in response:
                return True
            if "Error: " in response:
                return False
        except Exception as e:
            print(f"Error: {e}")
            break
    return False

status = remove_file(serial_port, "test.pdf")
print(status)

Device -> PC: Success: Removed file test.pdf.
True


In [9]:
def restart_file_transfer(serial_port):
    header = {
        "restart": True,
    }
    message = (json.dumps(header) + "\n").encode()
    serial_port.write(message)
    while True:
        try:
            response = serial_port.readline().decode().strip()
            print("Device -> PC:", response)
            if "Success: Restarting file transfer" in response:
                return True
            if "Error: " in response:
                return False
        except Exception as e:
            print(f"Error: {e}")
            break
    return False


restart_file_transfer(serial_port)

Device -> PC: Success: Restarting file transfer...


True

In [10]:
def reboot(serial_port) -> bool:
    header = {
        "reset": True,
    }
    message = (json.dumps(header) + "\n").encode()
    serial_port.write(message)
    while True:
        try:
            response = serial_port.readline().decode().strip()
            print("Device -> PC:", response)
            if "Info: Firmware update complete." in response:
                return True
            if "Error: " in response:
                return False
        except Exception as e:
            print(f"Error: {e}")
            break
    return False

In [11]:
def send_file(serial_port, file_path) -> bool:
    restart_file_transfer(serial_port)
    available_space, _ = request_disc_available_space(serial_port)
    free_memory, _ = request_memory(serial_port)
    CHUNK_SIZE = int(free_memory * 0.1)

    # Read the file to be transferred
    with open(file_path, "rb") as f:
        file_content = f.read()
    file_size = len(file_content)
    if file_size > available_space:
        print(f"Error: Not enough space on the device to transfer {file_path}")
        return False
    # resize CHUNK_SIZE to the power of 2
    CHUNK_SIZE = 2 ** (CHUNK_SIZE.bit_length() - 1)
    if file_size < CHUNK_SIZE:
        CHUNK_SIZE = file_size
    # Split the file into chunks
    chunks = [file_content[i : i + CHUNK_SIZE] for i in range(0, file_size, CHUNK_SIZE)]
    print(
        f"file size {file_size} bytes, select chunk size of {CHUNK_SIZE} bytes, {len(chunks)} chunks total"
    )

    # Send header message with the filename
    default_header = {
        "filename": os.path.basename(file_path),
        "finish": False,
    }

    # Send each chunk as a JSON message
    for i, chunk in enumerate(chunks):
        # Encode the chunk in base64
        chunk_b64 = base64.b64encode(chunk).decode()
        # Create a JSON message with the chunk info
        chunk_msg = default_header.copy()
        chunk_msg["chunk_size"] = str(len(chunk))
        chunk_msg["chunk_data_b64"] = chunk_b64
        # if this is the last chunk, set finish to True
        if i == len(chunks) - 1:
            checksum = hashlib.sha256(file_content).hexdigest()
            chunk_msg["checksum"] = checksum
            chunk_msg["size"] = file_size
            chunk_msg["finish"] = True
            # print("Sent finish message")
        message = (json.dumps(chunk_msg) + "\n").encode()
        serial_port.write(message)
        print(f"PC -> Device: Sent chunk {i} of size {len(chunk)} bytes")
        # read the response from the device, block reading
        while True:
            try:
                response = serial_port.readline().decode().strip()
                print("Device -> PC:", response)
                if "Success: finished receiving" in response:
                    return True
                if "Success: Received chunk" in response:
                    break
                if "Error: " in response:
                    return False
            except Exception as e:
                print(f"Error: {e}")
                break
    return False

In [12]:
send_file(serial_port, "bootloader.py")

Device -> PC: Success: Restarting file transfer...
Device -> PC: Info: Available space on the disk: 1351680 bytes, total space: 1441792 bytes.
Device -> PC: Info: free Memory 219584 bytes, total Memory 233024 bytes.
file size 7464 bytes, select chunk size of 7464 bytes, 1 chunks total
PC -> Device: Sent chunk 0 of size 7464 bytes
Device -> PC: Info: Starting a new file update: bootloader.py
Device -> PC: Success: finished receiving bootloader.py of size 7464 bytes.


True

In [13]:
send_file(serial_port, "test.pdf")

Device -> PC: Success: Restarting file transfer...
Device -> PC: Info: Available space on the disk: 1351680 bytes, total space: 1441792 bytes.
Device -> PC: Info: free Memory 201920 bytes, total Memory 233024 bytes.
file size 1052542 bytes, select chunk size of 16384 bytes, 65 chunks total
PC -> Device: Sent chunk 0 of size 16384 bytes
Device -> PC: Info: Starting a new file update: test.pdf
Device -> PC: Success: Received chunk of size 16384, total received 16384 bytes.
PC -> Device: Sent chunk 1 of size 16384 bytes
Device -> PC: Success: Received chunk of size 16384, total received 32768 bytes.
PC -> Device: Sent chunk 2 of size 16384 bytes
Device -> PC: Success: Received chunk of size 16384, total received 49152 bytes.
PC -> Device: Sent chunk 3 of size 16384 bytes
Device -> PC: Success: Received chunk of size 16384, total received 65536 bytes.
PC -> Device: Sent chunk 4 of size 16384 bytes
Device -> PC: Success: Received chunk of size 16384, total received 81920 bytes.
PC -> Device

True

In [14]:
status = remove_file(serial_port, "test.pdf")

Device -> PC: Success: Removed file test.pdf.


In [15]:
serial_port.close()